# OR 2-2, something with football i guess

In [73]:
import numpy as np
import pandas as pd

import os

In [74]:
data_files = os.listdir("data")
data = pd.read_csv(os.path.join("data", data_files[0]))

for data_file in data_files[1:]:
    data = pd.concat([data, pd.read_csv(os.path.join("data", data_file))])

# Remove whitespaces
data.loc[:, "HomeTeam"] = data["HomeTeam"].str.strip()
data.loc[:, "AwayTeam"] = data["AwayTeam"].str.strip()
data

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Unnamed: 58,Unnamed: 59,Unnamed: 60
0,N1,29/08/08,Vitesse,Groningen,0.0,4.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N1,30/08/08,Nijmegen,Graafschap,2.0,0.0,H,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N1,30/08/08,Roda JC,Twente,1.0,1.0,D,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N1,30/08/08,Utrecht,PSV Eindhoven,1.0,5.0,A,0.0,3.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N1,30/08/08,Willem II,Ajax,2.0,1.0,H,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,N1,06/05/12,Heracles,Nijmegen,1.0,2.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,N1,06/05/12,NAC Breda,Waalwijk,3.0,2.0,H,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,N1,06/05/12,Roda,Utrecht,1.0,3.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,N1,06/05/12,Vitesse,Ajax,1.0,3.0,A,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
scores = data.loc[:, ["HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "Date"]].set_index(["HomeTeam", "AwayTeam"]).rename({"FTHG": "HomeGoals", "FTAG": "AwayGoals", "FTR": "HomeWin"}, axis=1)
scores["AwayWin"] = scores["HomeWin"] == "A"
scores["HomeWin"] = scores["HomeWin"] == "H"
scores["Draw"] = (scores["HomeWin"] == False) & (scores["AwayWin"] == False)

scores = scores.dropna()
new_index = list(scores.index)

for i, v in enumerate(scores.iterrows()):
    clubs = v[0]
    res = v[1]
    if clubs[0] > clubs[1]:
        new_index[i] = (clubs[1], clubs[0])
        scores.iloc[i] = (res["AwayGoals"], res["HomeGoals"], res["AwayWin"], res["Date"], res["HomeWin"], res["Draw"])

scores.index = pd.MultiIndex.from_tuples(new_index)
scores = scores.rename({"HomeGoals": "FirstGoals", "AwayGoals": "SecondGoals", "HomeWin": "FirstWin", "AwayWin": "SecondWin"}, axis=1)
scores

,,FirstGoals,SecondGoals,FirstWin,Date,SecondWin,Draw
Groningen,Vitesse,4.0,0.0,True,29/08/08,False,False
Graafschap,Nijmegen,0.0,2.0,False,30/08/08,True,False
Roda JC,Twente,1.0,1.0,False,30/08/08,False,True
PSV Eindhoven,Utrecht,5.0,1.0,True,30/08/08,False,False
Ajax,Willem II,1.0,2.0,False,30/08/08,True,False
...,...,...,...,...,...,...,...
Heracles,Nijmegen,1.0,2.0,False,06/05/12,True,False
NAC Breda,Waalwijk,3.0,2.0,True,06/05/12,False,False
Roda,Utrecht,1.0,3.0,False,06/05/12,True,False
Ajax,Vitesse,3.0,1.0,True,06/05/12,False,False


We can define the weights as follows:
$f_{i, j} = \frac{l_{i, j} + 0.5d_{i, j}}{2g_{i, j}} + \frac{c_{i, j}}{2c_{i, j} + 2s_{i, j}}$,
where $l_{i, j}$ is games lost, $l_{i, j}$ draw, $g_{i, j}$  is total games. $c_{i, j}$ is the total goals conceded, $s_{i, j}$ is total goals scored. So first we need to get all those statistics. Note: $f_{i, j} + f_{j, i} = 1$.

In [78]:
stats = scores.groupby(scores.index).sum()
number_of_games = scores.groupby(scores.index).size().to_frame("GamesNumber")
stats = stats.merge(number_of_games, left_index=True, right_index=True)
stats.index = pd.MultiIndex.from_tuples(stats.index)
stats

FirstGoals  SecondGoals  FirstWin  SecondWin  Draw  \
AZ Alkmaar Ajax             30.0         56.0         8         14     5   
           Cambuur          14.0          5.0         5          0     3   
           Den Haag         58.0         23.0        18          5     3   
           Dordrecht         5.0          1.0         2          0     0   
           Excelsior        25.0         21.0         5          3     6   
...                          ...          ...       ...        ...   ...   
Vitesse    Zwolle           40.0         20.0        14          4     2   
Volendam   Willem II         4.0          3.0         1          0     1   
Waalwijk   Willem II         5.0         13.0         0          7     2   
           Zwolle            8.0         13.0         0          2     7   
Willem II  Zwolle           16.0         21.0         6          9     3   

                      GamesNumber  
AZ Alkmaar Ajax                27  
           Cambuur              8  
           Den Haag            26  
           Dordrecht            2  
           Excelsior           14  
...                           ...  
Vitesse    Zwolle              20  
Volendam   Willem II            2  
Waalwijk   Willem II            9  
           Zwolle               9  
Willem II  Zwolle              18  

[360 rows x 6 columns]

In [79]:
weights_series = (stats["SecondWin"] + 0.5*stats["Draw"])/(2 * stats["GamesNumber"]) + (stats["SecondGoals"]/(2*stats["FirstGoals"] + 2*stats["SecondGoals"]))

weights_copy = weights_series.copy()
second_index = pd.MultiIndex.from_arrays([weights_copy.index.get_level_values(1), weights_copy.index.get_level_values(0)])
weights_copy.index = second_index
weights_copy = 1 - weights_copy

weights = pd.concat([weights_series, weights_copy]).sort_index().unstack()
# weights.replace(np.nan, 0, inplace=True)
weights

,AZ Alkmaar,Ajax,Cambuur,Den Haag,Dordrecht,Excelsior,FC Emmen,Feyenoord,For Sittard,Go Ahead Eagles,...,Sparta,Sparta Rotterdam,Twente,Utrecht,VVV Venlo,Vitesse,Volendam,Waalwijk,Willem II,Zwolle
AZ Alkmaar,NaN,0.631137,0.225329,0.266975,0.083333,0.442547,0.033333,0.552363,0.170330,0.200893,...,0.000000,0.326389,0.435000,0.453661,0.240000,0.389676,0.000000,0.210737,0.365090,0.248051
Ajax,0.368863,NaN,0.119048,0.194919,0.392857,0.139488,0.038462,0.263997,0.048387,0.083333,...,0.401786,0.106061,0.277436,0.423521,0.057650,0.318681,0.166667,0.060897,0.183342,0.138158
Cambuur,0.774671,0.880952,NaN,0.633333,0.225000,0.477273,NaN,0.888889,1.000000,0.165385,...,NaN,0.208333,0.690476,0.442888,NaN,0.759524,NaN,0.283333,0.585714,0.492944
Den Haag,0.733025,0.805081,0.366667,NaN,0.125000,0.288177,0.746154,0.592727,0.700000,0.541667,...,0.545833,0.607337,0.600028,0.583791,0.428261,0.533582,0.000000,0.432870,0.331263,0.627101
Dordrecht,0.916667,0.607143,0.775000,0.875000,NaN,0.291667,NaN,0.900000,NaN,0.291667,...,NaN,NaN,1.000000,0.909091,NaN,0.909091,NaN,NaN,0.928571,0.928571
Excelsior,0.557453,0.860512,0.522727,0.711823,0.708333,NaN,0.166667,0.748626,0.732143,0.550000,...,NaN,0.268382,0.500000,0.625000,0.452083,0.623754,NaN,0.500000,0.428922,0.788889
FC Emmen,0.966667,0.961538,NaN,0.253846,NaN,0.833333,NaN,0.776667,0.446429,NaN,...,NaN,0.620192,0.633929,0.433333,0.416667,0.675439,NaN,0.464286,0.566667,0.696154
Feyenoord,0.447637,0.736003,0.111111,0.407273,0.100000,0.251374,0.223333,NaN,0.309524,0.258571,...,0.450000,0.204861,0.385913,0.337173,0.250355,0.382969,0.375000,0.262443,0.226751,0.341228
For Sittard,0.829670,0.951613,0.000000,0.300000,NaN,0.267857,0.553571,0.690476,NaN,0.785714,...,NaN,0.600000,0.480769,0.736111,0.357143,0.664835,NaN,0.372222,0.589286,0.476190
Go Ahead Eagles,0.799107,0.916667,0.834615,0.458333,0.708333,0.450000,NaN,0.741429,0.214286,NaN,...,NaN,0.619048,0.656250,0.598214,NaN,0.698052,NaN,0.480769,0.633333,0.464286


Hier gebruik ik die formule uit die 2e paper volgens mij.

In [92]:
damping_factor = 0.05 # geen idee waarom dit, maar is wel prima denk ik
weights_sum_df = weights.sum(axis=1)
sum_of_weights_matrix = np.array([weights_sum_df])
sum_of_weights_df = pd.DataFrame(np.repeat(sum_of_weights_matrix, len(weights_sum_df), axis=0))
sum_of_weights_df.index = list(weights_sum_df.index)
sum_of_weights_df.columns = list(weights_sum_df.index)
number_of_clubs = len(sum_of_weights_df.index)
cleaned_weights = weights.replace(np.nan, 0)
transition_probability_matrix = (1 - damping_factor) * cleaned_weights/sum_of_weights_df.transpose() + damping_factor/number_of_clubs
transition_probability_matrix

,AZ Alkmaar,Ajax,Cambuur,Den Haag,Dordrecht,Excelsior,FC Emmen,Feyenoord,For Sittard,Go Ahead Eagles,...,Sparta,Sparta Rotterdam,Twente,Utrecht,VVV Venlo,Vitesse,Volendam,Waalwijk,Willem II,Zwolle
AZ Alkmaar,0.001724,0.072140,0.026864,0.031510,0.011022,0.051099,0.005443,0.063351,0.020728,0.024138,...,0.001724,0.038139,0.050257,0.052339,0.028501,0.045200,0.001724,0.025236,0.042457,0.029399
Ajax,0.064541,0.001724,0.021998,0.034919,0.068627,0.025479,0.008274,0.046683,0.009964,0.015916,...,0.070148,0.019786,0.048971,0.073849,0.011542,0.055995,0.030107,0.012095,0.032947,0.025252
Cambuur,0.056439,0.063946,0.001724,0.046457,0.017616,0.035434,0.001724,0.064507,0.072354,0.013405,...,0.001724,0.016439,0.050493,0.033005,0.001724,0.055370,0.001724,0.021736,0.043093,0.036541
Den Haag,0.048127,0.052688,0.024935,0.001724,0.009637,0.019967,0.048958,0.039245,0.046036,0.036013,...,0.036277,0.040170,0.039708,0.038680,0.028834,0.035501,0.001724,0.029126,0.022694,0.041421
Dordrecht,0.070628,0.047362,0.059979,0.067496,0.001724,0.023648,0.001724,0.069375,0.001724,0.023648,...,0.001724,0.001724,0.076892,0.070058,0.001724,0.070058,0.001724,0.001724,0.071523,0.071523
Excelsior,0.037493,0.056939,0.035265,0.047399,0.047175,0.001724,0.012418,0.049760,0.048703,0.037015,...,0.001724,0.018945,0.033807,0.041828,0.030732,0.041748,0.001724,0.033807,0.029246,0.052344
FC Emmen,0.077669,0.077266,0.001724,0.021667,0.001724,0.067194,0.001724,0.062742,0.036797,0.001724,...,0.001724,0.050449,0.051528,0.035768,0.034459,0.054789,0.001724,0.038200,0.046244,0.056417
Feyenoord,0.048931,0.079341,0.013442,0.044674,0.012270,0.028233,0.025276,0.001724,0.034366,0.028993,...,0.049180,0.023328,0.042422,0.037282,0.028126,0.042111,0.041271,0.029401,0.025637,0.037709
For Sittard,0.061897,0.070741,0.001724,0.023482,0.001724,0.021151,0.041872,0.051801,0.001724,0.058709,...,0.001724,0.045240,0.036592,0.055111,0.027626,0.049942,0.001724,0.028720,0.044463,0.036260
Go Ahead Eagles,0.057075,0.065218,0.059534,0.033471,0.050787,0.032894,0.001724,0.053080,0.016567,0.001724,...,0.001724,0.044603,0.047180,0.043160,0.001724,0.050075,0.001724,0.035025,0.045592,0.033883


In [111]:
c = np.array(transition_probability_matrix)

accuracy = 5
while True:
    result = c @ c
    if (result.round(accuracy)[0] == result.round(accuracy)[1]).all():
        break

    c = result

ranking = pd.DataFrame(result[0], index=transition_probability_matrix.index, columns=["pagerank"])
ranking.sort_values(["pagerank"], ascending=False)

,pagerank
Ajax,0.060092
PSV Eindhoven,0.056871
Feyenoord,0.051620
AZ Alkmaar,0.051200
Twente,0.046939
Vitesse,0.043931
Utrecht,0.043320
Groningen,0.042212
Heerenveen,0.041486
Heracles,0.036789
